# 0. 데이터 불러오기

In [11]:
import pandas as pd
df = pd.read_csv("Dataset_03.csv")
df.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female


# 1. 이상치 분류

In [12]:
df_1 = df.copy()

In [13]:
df_1["forehead_ratio"] = df_1["forehead_width_cm"] / df_1["forehead_height_cm"]
df_1.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender,forehead_ratio
0,1,11.8,6.1,1,0,1,1,Male,1.934426
1,0,14.0,5.4,0,0,1,0,Female,2.592593


In [14]:
mean = df_1["forehead_ratio"].mean()
std = df_1["forehead_ratio"].std()
print(mean, std)

2.233618718921447 0.26452529699735255


In [15]:
condi_1 = df_1["forehead_ratio"] < (mean - 3 * std)
condi_2 = df_1["forehead_ratio"] > (mean + 3 * std)

In [16]:
(condi_1 | condi_2).sum()

3

- 정답 : 3

# 2. 통계 검정

In [17]:
df_2 = df_1.copy()
df_2.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender,forehead_ratio
0,1,11.8,6.1,1,0,1,1,Male,1.934426
1,0,14.0,5.4,0,0,1,0,Female,2.592593


In [18]:
from scipy.stats import ttest_ind

In [19]:
stat, p = ttest_ind(df_2.loc[df_2["gender"] == "Male", "forehead_ratio"],
         df_2.loc[df_2["gender"] == "Female", "forehead_ratio"], equal_var = False)
print(stat, p)

2.9994984197511543 0.0027186702390657176


In [20]:
round(stat, 3)

2.999

< 정답 >
- 검정통계량의 추정치 : 2.999
- 검정 결과 : p-value 가 0.05 미만이므로, 귀무가설을 기각.</br>
    - 따라서 성별에 따라 forehead_ratio 평균에 유의미한 차이가 있는 것으로 보인다.

# 3. 로지스틱 회귀분석

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [22]:
df_3 = df.copy()
df_3.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
0,1,11.8,6.1,1,0,1,1,Male
1,0,14.0,5.4,0,0,1,0,Female


In [23]:
df_train, df_test = train_test_split(df_3, train_size=0.7, random_state=123)
print(len(df_train), len(df_test))

3500 1501


In [24]:
model = LogisticRegression().fit(X = df_train.drop("gender", axis = 1), y = df_train["gender"])

In [ ]:
LogisticRegression().fit()

In [25]:
df_test.head(2)

,long_hair,forehead_width_cm,forehead_height_cm,nose_wide,nose_long,lips_thin,distance_nose_to_lip_long,gender
2648,0,11.7,6.8,1,1,0,1,Male
2456,1,14.6,7.0,1,1,1,1,Male


In [26]:
pred = model.predict_proba(df_test.drop("gender", axis = 1))
pred[:4, ]

array([[2.13366044e-02, 9.78663396e-01],
       [8.97496054e-05, 9.99910250e-01],
       [2.54440411e-01, 7.45559589e-01],
       [2.15025315e-03, 9.97849747e-01]])

In [27]:
result = pred[:, 1] > 0.5
result

array([ True,  True,  True, ..., False,  True,  True])

In [28]:
ans = (df_test["gender"] == "Male")
ans

2648     True
2456     True
4558     True
4885     True
92       True
        ...  
734      True
452     False
2558    False
1927     True
4037     True
Name: gender, Length: 1501, dtype: bool

In [29]:
precision = metrics.precision_score(y_true = ans, y_pred = result)

In [30]:
round(precision, 2)

0.96

- 정답 : 0.96